<a href="https://colab.research.google.com/github/bird0401/image_search/blob/main/no_metric_learning%20/1_get_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get model from Tensorflow Hub

In [1]:
# get mobilenet_v3
!wget --output-document mobilenet_v3_large_100_224_feature_vector_v5.tar.gz "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5?tf-hub-format=compressed"

--2022-05-12 16:09:09--  https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5?tf-hub-format=compressed
Resolving tfhub.dev (tfhub.dev)... 142.251.10.139, 142.251.10.113, 142.251.10.102, ...
Connecting to tfhub.dev (tfhub.dev)|142.251.10.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-modules/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5.tar.gz [following]
--2022-05-12 16:09:10--  https://storage.googleapis.com/tfhub-modules/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15840597 (15M) [application/octet-stream]
Saving to: ‘mobilenet_v3_large_100_224_feature_vector_v5.tar.gz’

mobilenet_v3_large_ 100%[

In [2]:
!ls -l mobilenet_v3_large_100_224_feature_vector_v5.tar.gz

-rw-r--r-- 1 root root 15840597 Feb  9  2021 mobilenet_v3_large_100_224_feature_vector_v5.tar.gz


In [3]:
# confirm whether there is no broken data
!sha1sum mobilenet_v3_large_100_224_feature_vector_v5.tar.gz

b4181065be4258956e249ea56e27cbeb8306372b  mobilenet_v3_large_100_224_feature_vector_v5.tar.gz


In [4]:
!mkdir mobilenet_v3_large_100_224_feature_vector_v5
!tar zxfv mobilenet_v3_large_100_224_feature_vector_v5.tar.gz -C mobilenet_v3_large_100_224_feature_vector_v5

In [6]:
!python3 -m pip install tf2onnx

     |████████████████████████████████| 440 kB 31.0 MB/s 
     |████████████████████████████████| 12.8 MB 66.6 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


# Convert to ONNX

In [7]:
!python3 -m tf2onnx.convert --saved-model mobilenet_v3_large_100_224_feature_vector_v5 --output mobilenet_v3_large_100_224_feature_vector_v5.onnx

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-05-12 16:09:26,830 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-05-12 16:09:28,329 - INFO - Signatures found in model: [serving_default].
2022-05-12 16:09:28,329 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-05-12 16:09:28,330 - INFO - Output names: ['feature_vector']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-05-12 16:09:30,008 - WARNING - From /usr/local/lib/python3.7/dist-packages/tf2onnx/tf_loader.py:711: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-05-12 16:09:30,583 - INFO - Us

In [ ]:
!ls -l mobilenet_v3_large_100_224_feature_vector_v5.onnx

In [8]:
!sha1sum mobilenet_v3_large_100_224_feature_vector_v5.onnx

df8c22fd55898d5325ab501cab2362da95ad3149  mobilenet_v3_large_100_224_feature_vector_v5.onnx


# inference

In [9]:
!nvidia-smi | head -n 12

Thu May 12 16:09:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W /  70W |      0MiB / 15109MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+


In [12]:
!pip install onnxruntime-gpu==1.7.0

     |████████████████████████████████| 29.9 MB 1.2 MB/s 


In [13]:
import numpy as np
import onnxruntime

In [14]:
onnxruntime.get_device()

'GPU'

In [16]:
session = onnxruntime.InferenceSession("mobilenet_v3_large_100_224_feature_vector_v5.onnx")

In [17]:
session.get_providers()

['CUDAExecutionProvider', 'CPUExecutionProvider']

In [26]:
session.get_inputs()[0].name

'inputs'

In [28]:
session.get_outputs()[0].name 

'feature_vector'

In [29]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name 

In [30]:
session.run([output_name], {input_name:np.zeros((1, 224, 224, 3), dtype=np.float32)})

[array([[-0.19617254, -0.32971072,  0.01838558, ...,  1.1617097 ,
         -0.36636543, -0.33906448]], dtype=float32)]

In [31]:
session.run([output_name], {input_name: np.ones((1, 224, 224, 3), dtype=np.float32)})

[array([[ 0.46933305,  0.20881234,  0.306212  , ..., -0.3671668 ,
         -0.3148665 , -0.3328042 ]], dtype=float32)]

In [33]:
# !cp /content/mobilenet_v3_large_100_224_feature_vector_v5.onnx /content/drive/MyDrive/mnt/model